In [ ]:
# transformers版

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b-8k", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b-8k")

if torch.cuda.is_available():
    model = model.to("cuda")



You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [2]:
def inference(text):
    token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            max_new_tokens=1000,
            min_new_tokens=1000,
            do_sample=True,
            temperature=1.0,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(output_ids.tolist()[0])
    print(output)

In [4]:
text = "昔々あるところに、宇宙海賊ピーチボーイという"
inference(text)

昔々あるところに、宇宙海賊ピーチボーイという悪者によって惑星が侵略され、その惑星を救うためにピーチボーイの基地で仲間として戦うことになったのでございます。
そのピーチボーイが宇宙海賊をしていて宇宙の悪の帝王(悪の首領)であるボスが惑星と宇宙を自分の好きなようにしようとしているのでそれを退治するために、宇宙怪盗としてピーチボーイに雇われたヒーロー(ヒーローはピーチボーイを宇宙で戦って、ピーチボーイは宇宙でヒーローたちを戦います)が宇宙に散らばっている様々な星からピーチボーイのもとにやってきます。
この物語はヒーローたちがピーチボーイに雇われて宇宙の悪と対決するストーリーです。
ピーチボーイは宇宙の悪と戦いながらも宇宙怪盗としてはピーチボーイに雇われたヒーローたちを地球に派遣してピーチボーイの縄張りを荒らさせ、その縄張りで悪の悪者(悪のボス)と戦っているヒーローたちをピーチボーイに頼んで守らせ、ピーチボーイが縄張りで倒した悪の悪者(悪のボス)を地球に送ってヒーローたちの戦いを助けさせようとします。
悪の悪者にピーチボーイの縄張りを荒らされていることを知らされたピーチボーイは地球に行って悪の悪者(悪のボス)を退治するのですが、悪の悪者が悪の悪者(悪のボス)を地球の周りに集結してピーチボーイと対決しているのですが、それは地球の周りのピーチボーイの縄張りに悪の悪者が集結してピーチボーイが善のヒーローたちを守れないようにして、ピーチボーイが縄張りを侵略してきた悪者(悪の悪者のボス)を退治しているのをヒーローがピーチボーイを応援して悪の悪者(悪のボス)を退治するストーリーです。
悪の悪者に悪の悪者(悪のボス)を退治されて、地球はピーチボーイの縄張りに守られ善のヒーローたちを守れることがわかって、ピーチボーイは悪の悪者(悪のボス)を退治してヒーローたちを守りました。
ピーチボーイは正義のヒーローが悪の悪者(悪のボス)を退治してヒーローたちが地球を守るために悪と戦っているのを応援し、悪の悪の勢力を退治することでヒーローたちを守っていることがわかります。
宇宙海賊ピーチボーイ物語(全13巻)
宇宙海賊ピーチボーイ2(全10巻)
宇宙海賊ピーチボーイ(全14巻)
1981年12月12日〜12月19日に、日本テレビ系列の「火曜サスペンス劇場」で放映。
企画:渡辺由自(日本テレビ)
プロデュ

In [ ]:
# ct2版

In [1]:
!pip install ctranslate2
!pip install protobuf

In [10]:
!rm -rf rinna-bilingual-gpt-neox-4b-8k-ct2
!ct2-transformers-converter --model rinna/bilingual-gpt-neox-4b-8k --quantization int8_float16 --output_dir rinna-bilingual-gpt-neox-4b-8k-ct2
!du -sh ./*

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
16K	./8k.ipynb
12K	./minigpt-4.ipynb
3.6G	./rinna-bilingual-gpt-neox-4b-8k-ct2


In [2]:
import ctranslate2
import transformers

import torch


model = "rinna/bilingual-gpt-neox-4b-8k"
ct2_model= model.replace("/", "-") + "-ct2"

# cudaが使える場合はcudaを使う
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"USE: {device}")

# generator = ctranslate2.Generator("rinna_gpt_neox_ppo_ct2_ft16", device=device)
generator = ctranslate2.Generator(ct2_model, device=device)
tokenizer = transformers.AutoTokenizer.from_pretrained(model, use_fast=False, legacy=False)


# 返信を作成する
def inference(msg):
    p = msg
    tokens = tokenizer.convert_ids_to_tokens(
        tokenizer.encode(
            p,
            add_special_tokens=False,
        )
    )

    results = generator.generate_batch(
        [tokens],
        max_length=100,
        min_length=100,
        sampling_topk=0,
        sampling_temperature=1.0,
        include_prompt_in_result=True,
    )

    text = tokenizer.decode(results[0].sequences_ids[0])
    print("システム(rinnna-8k): " + text + "\n")
    return text

inference("昔々あるところに")

USE: cuda
システム(rinnna-8k): 昔々あるところに。。。。。;;わずともず地果てていることていることこの世<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<↓↓!!ささ☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆こんなどっちどっちどっちどっちどっちどっち



'昔々あるところに。。。。。;;わずともず地果てていることていることこの世<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<↓↓!!ささ☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆こんなどっちどっちどっちどっちどっちどっち'